<a href="https://colab.research.google.com/github/mitis1233/-RBQ/blob/main/so_vits_svc_4_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

该笔记本默认配置只训练单个说话人的模型！

In [ ]:
# 查看显卡
!nvidia-smi

Mon Jun  5 00:08:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title 克隆的github仓库
!git clone https://github.com/koharubiyori/so-vits-svc-4.0-backup so-vits-svc

Cloning into 'so-vits-svc'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 79 (delta 18), reused 75 (delta 14), pack-reused 0
Unpacking objects: 100% (79/79), 272.60 KiB | 1.12 MiB/s, done.


In [ ]:
#@title 将python版本修改为3.8
#@ 根据命令行的提示，点击"or type selection number"后面闪烁的光标后，输入python3.8前面的数字
!sudo update-alternatives --config python3
!python --version
!sudo apt install python3-pip

There are 2 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                 Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.10   2         auto mode
  1            /usr/bin/python3.10   2         manual mode
  2            /usr/bin/python3.8    1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in manual mode
Python 3.8.10
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python-pip-whl python3-setuptools python3-wheel
Suggested packages:
  python-setuptools-doc
The following NEW packages will be installed:
  python-pip-whl python3-pip python3-setuptools python3-wheel
0 upgraded, 4 newly installed, 0 to remove and 34 not upgraded.
Need to get 2,389 kB of archi

In [ ]:
#@title 安装依赖
#@markdown 安装完毕后可能会提示重启运行时(RESTART RUNTIME)，**无需重启**。
%cd /content/so-vits-svc
%pip install -r requirements.txt
%pip install numpy==1.23.0 --force-reinstall


/content/so-vits-svc
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 6.4 MB/s 
     |████████████████████████████████| 20.0 MB 1.2 MB/s 
     |████████████████████████████████| 15.4 MB 10 kB/s 
     |████████████████████████████████| 221 kB 78.0 MB/s 
     |████████████████████████████████| 39.3 MB 1.2 MB/s 
     |████████████████████████████████| 887.4 MB 1.6 kB/s 
     |████████████████████████████████| 4.2 MB 64.5 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 142 kB 67.1 MB/s 
     |████████████████████████████████| 10.7 MB 60.5 MB/s 
     |████████████████████████████████| 14.6 MB 62.8 MB/s 
     |████████████████████████████████| 2.2 MB 70.5 MB/s 
     |████████████████████████████████| 677 kB 79.6 MB/s 
     |████████████████████████████████| 11.0 MB 77.3 MB/s 
     |████████████████████████████████| 203 kB 71.5 MB/s 
     |█████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.1 MB 18.8 MB/s 
ERROR: scipy 1.7.3 has requirement numpy<1.23.0,>=1.16.5, but you'll have numpy 1.23.0 which is incompatible.
ERROR: scikit-image 0.21.0 has requirement scipy>=1.8, but you'll have scipy 1.7.3 which is incompatible.
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3


In [ ]:
#@title 下载必要模型文件
!wget -P hubert/ https://huggingface.co/Himawari00/so-vits-svc4.0-pretrain-models/resolve/main/checkpoint_best_legacy_500.pt

--2023-06-05 00:15:45--  https://huggingface.co/Himawari00/so-vits-svc4.0-pretrain-models/resolve/main/checkpoint_best_legacy_500.pt
Resolving huggingface.co (huggingface.co)... 18.155.68.38, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/81/65/816584b073abd851a20d09c07bfdc9eb3943d88c67447f08224566172951227d/60d936ec5a566776fc392e69ad8b630d14eb588111233fe313436e200a7b187b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27checkpoint_best_legacy_500.pt%3B+filename%3D%22checkpoint_best_legacy_500.pt%22%3B&Expires=1686183346&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzgxLzY1LzgxNjU4NGIwNzNhYmQ4NTFhMjBkMDljMDdiZmRjOWViMzk0M2Q4OGM2NzQ0N2YwODIyNDU2NjE3Mjk1MTIyN2QvNjBkOTM2ZWM1YTU2Njc3NmZjMzkyZTY5YWQ4YjYzMGQxNGViNTg4MTExMjMzZmUzMTM0MzZlMjAwYTdiMTg3Yj9yZXNwb25zZS1jb250ZW50

# 数据集预处理

In [ ]:
#@title 加载Google云端硬盘
#@markdown 加载Google云端硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title 填写位于谷歌云端硬盘的项目相关文件夹配置
#@markdown 从谷歌云盘加载打包好的数据集进行预处理，首先要将打包好的zip格式数据集文件放到项目相关文件夹下的dataset文件夹中，数据集文件名使用角色名称。

#@markdown 文件夹名称
GD_DIR = "so-vits-svc-4.0"  #@param {type:"string"}
#@markdown 角色名称，这个值和接下来要填写的数据集压缩包名相同
CHARA_NAME = "kafuu_chino"  #@param {type:"string"}

gd_full_path = "/content/drive/MyDrive/" + GD_DIR

In [ ]:
#@title 从谷歌云盘加载打包好的数据集进行预处理
zip_name = gd_full_path + "/dataset/" + CHARA_NAME

!unzip -d /content/so-vits-svc/dataset_raw {zip_name}.zip

Archive:  /content/drive/MyDrive/so-vits-svc-4.0/dataset/kafuu_chino.zip
   creating: /content/so-vits-svc/dataset_raw/kafuu_chino/
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(1).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(10).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(100).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(101).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(102).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(103).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(104).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(105).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(106).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(107).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(108).wav  
  inflating: /content/so-vits-svc/dataset_raw/kafuu_chino/(109).wav  
  inflating: /content/so-vits-s

In [ ]:
#@title 重采样到44k
!python resample.py

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
./dataset_raw/kafuu_chino
900it [00:10, 87.74it/s] 


In [ ]:
#@title 划分训练集 生成配置文件
!python preprocess_flist_config.py

  0% 0/1 [00:00<?, ?it/s]warning：文件名./dataset/44k/kafuu_chino/(60).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(593).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(26).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(236).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(223).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(688).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(383).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(293).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(147).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(298).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(96).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(285).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dataset/44k/kafuu_chino/(362).wav中包含非字母数字下划线，可能会导致错误。（也可能不会）
warning：文件名./dat

In [ ]:
#@title 生成hubert和f0
!python preprocess_hubert_f0.py

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[900]
/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Loading hubert for content...
load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is /content/so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False,

In [ ]:
#@title 至此数据集预处理制作完毕，将数据集和相关文件保存到谷歌云盘项目文件夹下的trainingBackup文件夹，方便下次训练使用
#压缩dataset文件夹
!zip -r dataset.zip /content/so-vits-svc/dataset

gd_backup_path = gd_full_path + "/trainingBackup/" + CHARA_NAME
!mkdir -p {gd_backup_path}

!cp /content/so-vits-svc/dataset.zip {gd_backup_path}
!cp configs/config.json {gd_backup_path}
!cp filelists/train.txt {gd_backup_path}
!cp filelists/val.txt {gd_backup_path}

  adding: content/so-vits-svc/dataset/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/ (stored 0%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(690).wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(60).wav (deflated 35%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(867).wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(684).wav.soft.pt (deflated 8%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(52).wav.f0.npy (deflated 57%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(54).wav.f0.npy (deflated 76%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(593).wav (deflated 5%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(821).wav.f0.npy (deflated 27%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(26).wav (deflated 36%)
  adding: content/so-vits-svc/dataset/44k/kafuu_chino/(784).wav.soft.pt (deflat

In [ ]:
#@title 已经预处理过数据集的话，就可以跳过预处理部分，直接从云盘解压处理过的数据以及配置文件

gd_backup_path = gd_full_path + "/trainingBackup/" + CHARA_NAME

!unzip {gd_backup_path}/dataset.zip -d /
!cp {gd_backup_path}/config.json /content/so-vits-svc/configs/config.json
!cp {gd_backup_path}/val.txt filelists/val.txt
!cp {gd_backup_path}/train.txt filelists/train.txt

Archive:  /content/drive/MyDrive/so-vits-svc-4.0/trainingBackup/kafuu_chino/dataset.zip
   creating: /content/so-vits-svc/dataset/
   creating: /content/so-vits-svc/dataset/44k/
   creating: /content/so-vits-svc/dataset/44k/kafuu_chino/
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(690).wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(60).wav  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(867).wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(684).wav.soft.pt  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(52).wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(54).wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(593).wav  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(821).wav.f0.npy  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(26).wav  
  inflating: /content/so-vits-svc/dataset/44k/kafuu_chino/(784).wav.soft.pt  
  inflating: /

# 训练

In [ ]:
#@title  选择是否将训练后的模型保存到谷歌云盘 和 是否使用预模型
gd_backup_logs_path = gd_full_path + "/trainingBackup/" + CHARA_NAME + "/logs/44k"

#@markdown **将训练后的模型文件保存到谷歌云盘，勾选后，恢复训练时也需要勾选并执行**
SAVE_TO_DRIVE = True #@param {type:"boolean"}
if SAVE_TO_DRIVE:
  !mkdir -p /content/so-vits-svc/logs
  !mkdir -p {gd_backup_logs_path}
  !ln -s {gd_backup_logs_path}/ /content/so-vits-svc/logs

#@markdown **首次训练下载预模型 之后继续训练则使用自己保存的记录点，无需再下载**

#@markdown **使用预模型，下面打钩自动下载并启用**
PRE_PTH = False #@param {type:"boolean"}
if PRE_PTH:
  !wget -P logs/44k/ https://huggingface.co/Himawari00/so-vits-svc4.0-pretrain-models/resolve/main/G_0.pth
  !wget -P logs/44k/ https://huggingface.co/Himawari00/so-vits-svc4.0-pretrain-models/resolve/main/D_0.pth


In [ ]:
!rm -r /content/so-vits-svc/logs

In [ ]:
#@title  开始训练

#@markdown **启用tensorboard可视化数据**
TENSORBOARD_ON = False #@param {type:"boolean"}
if TENSORBOARD_ON:
  %load_ext tensorboard
  %tensorboard --logdir logs/44k

!python train.py -c configs/config.json -m 44k


/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
INFO:44k:{'train': {'log_interval': 200, 'eval_interval': 800, 'seed': 1234, 'epochs': 10000, 'learning_rate': 0.0002, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 16, 'fp16_run': False, 'lr_decay': 0.999875, 'segment_size': 10240, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0, 'use_sr': True, 'max_speclen': 512, 'port': '8001', 'keep_ckpts': 3}, 'data': {'training_files': 'filelists/train.txt', 'validation_files': 'filelists/val.txt', 'max_wav_value': 32768.0, 'sampling_ra

In [ ]:
#@title 训练聚类模型
!python cluster/train_cluster.py

/usr/local/lib/python3.8/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
train kmeans for kafuu_chino...
INFO:__main__:Loading features from dataset/44k/kafuu_chino
43it [00:00, 364.46it/s]
Traceback (most recent call last):
  File "cluster/train_cluster.py", line 64, in <module>
    x = train_cluster(in_dir, n_clusters, verbose=False)
  File "cluster/train_cluster.py", line 22, in train_cluster
    features.append(torch.load(path).squeeze(0).numpy().T)
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 789, in load
    return _load(opened_zipfile, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", line 1131, in _load
    result = unpickler.load()
  File "/usr/local/lib/python3.8/dist-packages/torch/serialization.py", 

In [ ]:
#@title 手动将训练后的最新模型文件备份到谷歌云盘（注意即使不进行手动备份，谷歌云盘里的44k文件夹下也会有训练时的模型文件）
import os, re
latest_check_point_name = sorted(i for i in os.listdir('./logs/44k') if re.match(r'^G_\d+\.pth$', i) != None)[-1]
latest_iteration_num = re.sub(r'^G_(\d+).pth$', r'\1', latest_check_point_name)

#@markdown ### 存储路径:
SAVE_PATH = '/content/drive/MyDrive/models/'  #@param {type:"string"}

!cp /content/so-vits-svc/logs/44k/G_"{latest_iteration_num}".pth "{SAVE_PATH}"
!cp /content/so-vits-svc/logs/44k/D_"{latest_iteration_num}".pth "{SAVE_PATH}"
!cp /content/so-vits-svc/configs/config.json "{SAVE_PATH}"

cluster_file_path = "/content/so-vits-svc/logs/44k/kmeans_10000.pt"
if os.path.exists(cluster_file_path):
  !cp "{cluster_file_path}" "{SAVE_PATH}"


cp: cannot create regular file '/content/drive/MyDrive/models/': Not a directory
cp: cannot create regular file '/content/drive/MyDrive/models/': Not a directory
cp: cannot create regular file '/content/drive/MyDrive/models/': Not a directory
cp: cannot create regular file '/content/drive/MyDrive/models/': Not a directory


In [ ]:
#@title 使用训练结果中的最新模型推理，**将要推理的音频置于so-vits-svc/raw中**
#@markdown ## 更多参数见：so-vits-svc/inference_main.py

#@markdown #### 音高调整，支持正负（半音）:
TRANS = 0 #@param {type:"integer"}
#@markdown #### 合成目标说话人名称（多人使用半角空格隔开）:
SPK_LIST = 'mayano' #@param {type:"string"}
#@markdown #### 语音转换自动预测音高，转换歌声时不要打开这个会严重跑调
AUTO_PREDICT_F0 = False #@param {type:"boolean"}
#@markdown #### 聚类方案占比，范围0-1，若没有训练聚类模型应使用0:
CLUSTER_INFER_RATIO = 0 #@param {type:"number"}


!python inference_main.py --trans "{TRANS}" --spk_list "{SPK_LIST}" --auto_predict_f0 "{AUTO_PREDICT_F0}" --cluster_infer_ratio "{CLUSTER_INFER_RATIO}"

Traceback (most recent call last):
  File "/content/so-vits-svc/inference_main.py", line 11, in <module>
    from inference import infer_tool
  File "/content/so-vits-svc/inference/infer_tool.py", line 13, in <module>
    import parselmouth
ModuleNotFoundError: No module named 'parselmouth'
